Update the code with proper directory/file names and parameters before running. Do not restrict your analyses to this workflow, explore your microbiota.

### STEP1:  Primers removal/trim-paired end reads

In [ ]:
%%bash
#
# Trim forward and reverse complementar reads in N bp
#

TRIM_12=51
TRIM_37=1
TRIM_DIR=Results/trimmed_seqs/all_seqs
SAMPLE_DIR_12=Samples/Seq1-2
SAMPLE_DIR_37=Samples/Seq3-7

mkdir -p $TRIM_DIR

#trim separately samples from seq1-2 and seq3-7
#for d in `find $SAMPLE_DIR_12 -name "*R1_001.fastq.gz"`; do
#	sampdir=`basename $d _L001_R1_001.fastq.gz`
#	mkdir -p $TRIM_DIR/$sampdir
#done

for f in Samples/Seq1-2/*; do #not trimmed 3' yet
    fastq=`basename $f`
    /home/users/vheidrich/seqtk/seqtk trimfq -e $TRIM_12 -b 21 $f | gzip -c > $TRIM_DIR/$fastq
done

#for d in `find $SAMPLE_DIR_37 -name "*R1_001.fastq.gz"`; do
#	sampdir=`basename $d _L001_R1_001.fastq.gz`
#	mkdir -p $TRIM_DIR/$sampdir
#done
for f in Samples/Seq3-7/*; do
    fastq=`basename $f`
    /home/users/vheidrich/seqtk/seqtk trimfq -e $TRIM_37 -b 21 $f | gzip -c > $TRIM_DIR/$fastq
done

In [ ]:
%%bash

TRIM_DIR=Results/trimmed_seqs/all_seqs

#Add a G to 0length reads

for f in $TRIM_DIR/*fastq.gz; do
    fastq=`basename $f`
    gunzip < $f | sed 's/^[[:blank:]]*$/G/' | gzip -c > temp.gz
    mv temp.gz $TRIM_DIR/$fastq
done

### STEP2: Import sequences into artifact

In [ ]:
%%bash
unset LD_LIBRARY_PATH
#
#In order to use QIIME 2, your input data must be stored in QIIME 2 artifacts
#
rm -r Results/trimmed_seqs/all_seqs/.ipynb_checkpoints #remove notebook checkpoints folder
TRIM_DIR=Results/trimmed_seqs

qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path $TRIM_DIR/all_seqs \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path $TRIM_DIR/demux-paired-end.qza

qiime demux summarize \
  --i-data $TRIM_DIR/demux-paired-end.qza \
  --o-visualization $TRIM_DIR/demux-paired-end.qzv

### STEP3: DADA2

In [ ]:
%%bash
unset LD_LIBRARY_PATH

#
# This method denoises paired-end sequences, dereplicates them, and filters
#  chimeras.The reads will also be joined.
#
#  --i-demultiplexed-seqs ARTIFACT PATH SampleData[PairedEndSequencesWithQuality]
#                                  The paired-end demultiplexed sequences to be
#                                  denoised.  [required]
#  --p-trunc-len-f INTEGER         Position at which forward read sequences
#                                  should be truncated due to decrease in
#                                  quality. This truncates the 3' end of the of
#                                  the input sequences, which will be the bases
#                                  that were sequenced in the last cycles.
#                                  Reads that are shorter than this value will
#                                  be discarded. After this parameter is
#                                  applied there must still be at least a 20
#                                  nucleotide overlap between the forward and
#                                  reverse reads.
#  --p-trunc-len-r INTEGER         Position at which reverse read sequences
#                                  should be truncated due to decrease in
#                                  quality. This truncates the 3' end of the of
#                                  the input sequences, which will be the bases
#                                  that were sequenced in the last cycles.
#                                  Reads that are shorter than this value will
#                                  be discarded. After this parameter is
#                                  applied there must still be at least a 20
#                                  nucleotide overlap between the forward and
#                                  reverse reads.
#  --o-table ARTIFACT PATH FeatureTable[Frequency]
#                                  The resulting feature table. 
#  --o-representative-sequences ARTIFACT PATH FeatureData[Sequence]
#                                  The resulting feature sequences. Each
#                                  feature in the feature table will be
#                                  represented by exactly one sequence, and
#                                  these sequences will be the joined paired-
#                                  end sequences.  [required if not passing
#                                  --output-dir]
#  --o-denoising-stats ARTIFACT PATH SampleData[DADA2Stats]
#                                  [required if not passing --output-dir]
#


DADAED_DIR=Results/trimmed_joined_dadaed_seqs
mkdir -p $DADAED_DIR

IARTIFACT=Results/trimmed_seqs/demux-paired-end.qza
OARTIFACT=$DADAED_DIR/demux-joined-dadaed-fs.qza
OARTIFACT1=$DADAED_DIR/demux-joined-dadaed-ft.qza
VARTIFACT=$DADAED_DIR/demux-joined-dadaed-stats.qza
VARTIFACT1=$DADAED_DIR/demux-joined-dadaed.qzv

#p-truncs based on demux-paired-end.qzv
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs $IARTIFACT \
  --p-trunc-len-f 0 \
  --p-trunc-len-r 0 \
  --p-max-ee-f 4.0 \
  --p-max-ee-r 4.0 \
  --o-representative-sequences $OARTIFACT \
  --p-n-threads 12 \
  --o-table $OARTIFACT1 \
  --verbose \
  --o-denoising-stats $VARTIFACT

#summarize feature table
qiime feature-table summarize \
    --i-table $OARTIFACT1 \
    --o-visualization $VARTIFACT1

#to view statistics
qiime tools export \
    --input-path $VARTIFACT \
    --output-path $DADAED_DIR

### STEP4: Chimeras identification and removal

In [ ]:
%%bash
unset LD_LIBRARY_PATH
#
# Apply the vsearch uchime_ref method to identify chimeric feature
# sequences. The results of this method can be used to filter chimeric
# features from the corresponding feature table.
#
#--i-sequences The feature sequences to be chimera-checked.
#--i-table Feature table
#--i-reference-sequences The non-chimeric reference sequences. 99% reference used
#--o-chimeras The chimeric sequences
#--o-nonchimeras The non-chimeric sequences
#--o-stats Summary statistics from chimera checking.
#

CHIMERAS_DIR=Results/trimmed_joined_dadaed_chimeric_seqs
DADAED_DIR=Results/trimmed_joined_dadaed_seqs
REF_DIR=/home/users/vheidrich/projects_current/Metagenomics/Silva/referenceOTUs_artifacts
mkdir -p $CHIMERAS_DIR

IARTIFACT=$DADAED_DIR/demux-joined-dadaed-fs.qza
IARTIFACT1=$DADAED_DIR/demux-joined-dadaed-ft.qza
IARTIFACT2=$REF_DIR/referenceOTUs_99.qza
OARTIFACT=$CHIMERAS_DIR/demux-joined-dadaed-chimeras-fs.qza
OARTIFACT1=$CHIMERAS_DIR/demux-joined-dadaed-nonchimera-fs.qza
VARTIFACT=$CHIMERAS_DIR/demux-joined-dadaed-chimeras_stats.qza
VARTIFACT1=$CHIMERAS_DIR/demux-joined-dadaed-chimeras-fs.qzv

#99
qiime vsearch uchime-ref \
    --i-sequences $IARTIFACT \
    --i-table $IARTIFACT1 \
    --i-reference-sequences $IARTIFACT2 \
    --o-chimeras $OARTIFACT \
    --o-nonchimeras $OARTIFACT1 \
    --p-threads 16 \
    --o-stats $VARTIFACT

qiime feature-table tabulate-seqs \
    --i-data $OARTIFACT \
    --o-visualization $VARTIFACT1

#qiime tools export \
#    --input-path $IARTIFACT1 \
#    --output-path $DADAED_DIR

#biom convert -i $DADAED_DIR/feature-table.biom -o $DADAED_DIR/feature-table.txt --to-tsv

In [ ]:
%%bash
unset LD_LIBRARY_PATH
#
#Filter features from table based on frequency and/or metadata.
#
#--i-table The feature table from which features should be filtered.
#--i-data The sequences from which features should be filtered.
#--m-metadata-file Metadata file or artifact viewable as metadata. This option may be supplied multiple 
#                  times to merge metadata. Feature metadata used with `where` parameter when selecting 
#                  features to retain, or with `exclude_ids` when selecting features to discard.
#--o-filtered-table The resulting filtered table.
#--o-filtered-data The resulting filtered sequences.
#--o-visualization Visual and tabular summaries of a feature table.
#

CHIMERIC_DIR=Results/trimmed_joined_dadaed_chimeric_seqs
CHIMERAS_DIR=Results/trimmed_joined_dadaed_chimeraremoved_seqs
DADAED_DIR=Results/trimmed_joined_dadaed_seqs
mkdir -p $CHIMERAS_DIR

IARTIFACT=$DADAED_DIR/demux-joined-dadaed-fs.qza
IARTIFACT1=$CHIMERIC_DIR/demux-joined-dadaed-nonchimera-fs.qza
IARTIFACT2=$DADAED_DIR/demux-joined-dadaed-ft.qza
OARTIFACT=$CHIMERAS_DIR/demux-joined-dadaed-chimeraremoved-ft.qza
OARTIFACT1=$CHIMERAS_DIR/demux-joined-dadaed-chimeraremoved-fs.qza
OARTIFACT2=$CHIMERAS_DIR/demux-joined-dadaed-chimeraremoved-ft.qzv

qiime feature-table filter-features \
  --i-table $IARTIFACT2 \
  --m-metadata-file $IARTIFACT1 \
  --o-filtered-table $OARTIFACT
  
qiime feature-table filter-seqs \
  --i-data $IARTIFACT \
  --m-metadata-file $IARTIFACT1 \
  --o-filtered-data $OARTIFACT1
  
qiime feature-table summarize \
  --i-table $OARTIFACT \
  --o-visualization $OARTIFACT2

#qiime tools export \
#    --input-path $OARTIFACT \
#    --output-path $CHIMERAS_DIR

#biom convert -i $CHIMERAS_DIR/feature-table.biom -o $CHIMERAS_DIR/feature-table.txt --to-tsv

### STEP5: Taxonomic assignment 

In [ ]:
%%bash
unset LD_LIBRARY_PATH
#
# explore the taxonomic composition of the samples and relate that to sample metadata.
#
#--i-classifier Naive Bayes classifier trained on Greengenes 99%
#--m-metadata-file mapping file
#--o-visualization output visualization path
#--i-reads OTUs representative sequences
#--i-table OTU table
#--o-clasification/--i-taxonomy/--i-input-file artifact with sequences assigned to taxonomy
#

REF_DIR=/home/users/vheidrich/projects_current/Metagenomics/Silva/referenceOTUs_artifacts
TAXA_DIR=Results/taxa_summary
#mkdir -p $TAXA_DIR

IARTIFACT=Results/trimmed_joined_dadaed_chimeraremoved_seqs/demux-joined-dadaed-chimeraremoved-fs.qza
OARTIFACT=$TAXA_DIR/taxonomy_vs.qza
VARTIFACT=$TAXA_DIR/taxonomy_vs.qzv
REF=$REF_DIR/referenceOTUs_99.qza
TAX=$REF_DIR/taxonomy_99.qza

#taxonomy assignment
qiime feature-classifier classify-consensus-vsearch \
  --i-query $IARTIFACT \
  --i-reference-reads $REF \
  --i-reference-taxonomy $TAX \
  --p-threads 180 \
  --o-classification $OARTIFACT

#generate a visualization of the resulting mapping from sequence to taxonomy
qiime metadata tabulate \
  --m-input-file $OARTIFACT \
  --o-visualization $VARTIFACT

### STEP6: Removal of non-bacterial ASVs 

In [ ]:
%%bash
unset LD_LIBRARY_PATH

ASV=Results/trimmed_joined_dadaed_chimeraremoved_seqs/demux-joined-dadaed-chimeraremoved-ft.qza
ASV_filtered=Results/trimmed_joined_dadaed_chimeraremoved_seqs/ASV_table_final.qza
qiime feature-table filter-features \
  --i-table $ASV \
  --m-metadata-file Results/taxa_summary/taxonomy_vs.qza \
  --p-where "Taxon='Unassigned' OR Taxon='D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Chloroplast' OR Taxon='D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rickettsiales;D_4__Mitochondria'" \
  --p-exclude-ids \
  --o-filtered-table $ASV_filtered
  
qiime feature-table summarize \
  --i-table $ASV_filtered \
  --o-visualization Results/trimmed_joined_dadaed_chimeraremoved_seqs/ASV_table_final.qzv

ASV=Results/trimmed_joined_dadaed_chimeraremoved_seqs/demux-joined-dadaed-chimeraremoved-fs.qza
ASV_filtered=Results/trimmed_joined_dadaed_chimeraremoved_seqs/ASV_seqs_final.qza

qiime feature-table filter-seqs \
  --i-data $ASV \
  --m-metadata-file Results/taxa_summary/taxonomy_vs.qza \
  --p-where "Taxon='Unassigned' OR Taxon='D_0__Bacteria;D_1__Cyanobacteria;D_2__Oxyphotobacteria;D_3__Chloroplast' OR Taxon='D_0__Bacteria;D_1__Proteobacteria;D_2__Alphaproteobacteria;D_3__Rickettsiales;D_4__Mitochondria'" \
  --p-exclude-ids \
  --o-filtered-data $ASV_filtered

### STEP7: Removal of <3,000 reads samples

In [ ]:
%%bash
unset LD_LIBRARY_PATH

ASV=Results/trimmed_joined_dadaed_chimeraremoved_seqs/ASV_table_final.qza
ASV_filtered=Results/trimmed_joined_dadaed_chimeraremoved_seqs/ASV_table_final_goodsamples.qza
qiime feature-table filter-samples \
  --i-table $ASV \
  --p-min-frequency 3000 \
  --o-filtered-table $ASV_filtered